##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Entrenamiento y servicio de un modelo de TensorFlow con TensorFlow Serving

**Advertencia: Este bloc de notas fue diseñado para ejecutarse únicamente en Google Colab**. Instala paquetes en el sistema y requiere acceso de raíz. Si desea ejecutarlo en un bloc de notas Jupyter local, hágalo con cuidado.

Nota: Puede ejecutar este ejemplo ahora mismo en un bloc de notas estilo Jupyter, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<tr>
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/serving/rest_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/serving/rest_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/serving/rest_simple.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/serving/rest_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</tr>
</table></div>

Esta guía entrena un modelo de red neuronal para clasificar [imágenes de indumentaria, como calzado deportivo y camisas](https://github.com/zalandoresearch/fashion-mnist), guarda el modelo entrenado y luego lo entrega con [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving). La atención se centra en TensorFlow Serving, en lugar del modelado y el entrenamiento en TensorFlow, por lo que para ver un ejemplo completo que se centre en el modelado y el entrenamiento, debe consultar el [ejemplo de clasificación básica](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/keras/basic_classification.ipynb).

Esta guía usa [tf.keras](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/keras.ipynb), una API de alto nivel que se usa para compilar y entrenar modelos en TensorFlow.

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major == 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

## Cómo crear un modelo

### Cómo importar el conjunto de datos Fashion MNIST

Esta guía usa el conjunto de datos [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) que contiene 70 000 imágenes en escala de grises en 10 categorías. Las imágenes muestran artículos individuales de prendas de vestir en baja resolución (28 x 28 píxeles), como se puede ver a continuación:

<table>
  <tr><td>     <img src="https://tensorflow.org/images/fashion-mnist-sprite.png" width="600" alt="Fashion MNIST sprite">
</td></tr>
  <tr><td align="center">     <b>Figura 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Muestras de Fashion-MNIST</a> (de Zalando, licencia MIT).<br> </td></tr>
</table>

Fashion MNIST fue pensado como reemplazo directo del clásico conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist/), a menudo usado como el "Hola, mundo" de los programas de aprendizaje automático para visión artificial. Puede acceder a Fashion MNIST directamente desde TensorFlow, simplemente importe y cargue los datos.

Nota: Aunque en realidad se trata de imágenes, se cargan como arreglos NumPy y no como objetos binarios de imagen.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

### Cómo entrenar y evaluar el modelo

Usemos la CNN más simple posible, ya que no nos centramos en la parte del modelado.

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))

## Cómo guardar el modelo

Para cargar nuestro modelo entrenado en TensorFlow Serving, primero debemos guardarlo en formato [SavedModel](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model). Esto generará un archivo protobuf en una jerarquía de directorios bien definida e incluirá un número de versión. [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) nos permite seleccionar qué versión de un modelo o "servible" queremos usar cuando enviamos solicitudes de inferencia. Cada versión se exportará a un subdirectorio diferente en la ruta indicada.

In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

## Cómo examinar el modelo guardado

Usaremos la utilidad de línea de comando `saved_model_cli` para ver [MetaGraphDefs](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/MetaGraphDef) (los modelos) y [SignatureDefs](https://www.tensorflow.org/tfx/serving/signature_defs) (los métodos que puede llamar) en nuestro SavedModel. Consulte [esta discusión sobre la CLI de SavedModel](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/saved_model.md#cli-to-inspect-and-execute-savedmodel) en la Guía de TensorFlow.

In [ ]:
!saved_model_cli show --dir {export_path} --all

¡Eso nos dice mucho sobre nuestro modelo! En este caso, simplemente entrenamos nuestro modelo, por lo que ya conocemos las entradas y salidas, pero si no lo hiciéramos, esta sería información importante. No nos dice todo, como el hecho de que se trata de datos de imágenes en escala de grises, por ejemplo, pero es un gran comienzo.

## Cómo servir el modelo con TensorFlow Serving

**Advertencia: Si NO está ejecutando esto en Google Colab,** las siguientes celdas instalarán paquetes en el sistema con acceso de raíz. Si desea ejecutarlo en un bloc de notas de Jupyter local, hágalo con cuidado.

### Cómo agregar el URI de distribución de TensorFlow Serving como fuente del paquete:

Nos estamos preparando para instalar TensorFlow Serving con [Aptitude](https://wiki.debian.org/Aptitude) ya que este Colab se ejecuta en un entorno Debian. Agregaremos el paquete `tensorflow-model-server` a la lista de paquetes que Aptitude conoce. Tenga en cuenta que estamos ejecutando como raíz.

Nota: Este ejemplo ejecuta TensorFlow Serving de forma nativa, pero [también puede ejecutarlo en un contenedor Docker](https://www.tensorflow.org/tfx/serving/docker), que es una de las formas más sencillas de comenzar a usar TensorFlow Serving.

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

### Cómo instalar TensorFlow Serving

Esto es todo lo que necesita: ¡una línea de comando!

In [ ]:
# TODO: Use the latest model server version when colab supports it.
#!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server
# We need to install Tensorflow Model server 2.8 instead of latest version
# Tensorflow Serving >2.9.0 required `GLIBC_2.29` and `GLIBCXX_3.4.26`. Currently colab environment doesn't support latest version of`GLIBC`,so workaround is to use specific version of Tensorflow Serving `2.8.0` to mitigate issue.
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-2.8.0/t/tensorflow-model-server/tensorflow-model-server_2.8.0_all.deb'
!dpkg -i tensorflow-model-server_2.8.0_all.deb
!pip3 install tensorflow-serving-api==2.8.0

### Cómo empezar a ejecutar TensorFlow Serving

Aquí es donde comenzamos a ejecutar TensorFlow Serving y cargamos nuestro modelo. Después de que se cargue, podemos comenzar a enviar solicitudes de inferencia usando REST. Hay algunos parámetros importantes:

- `rest_api_port`: el puerto que se usará para las solicitudes REST.
- `model_name`: lo usará en la URL de las solicitudes REST. Puede ser cualquier cosa.
- `model_base_path`: esta es la ruta al directorio donde se guardó el modelo.


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


In [ ]:
!tail server.log

## Cómo hacer una solicitud a su modelo en TensorFlow Serving

Primero, veamos un ejemplo aleatorio de nuestros datos de prueba.

In [ ]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))

Bien, eso se ve interesante. ¿Qué tan difícil resulta reconocerlo? Ahora creemos el objeto JSON para un lote de tres solicitudes de inferencia y veamos qué tan bien reconoce las cosas nuestro modelo:

In [ ]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

### Cómo hacer solicitudes REST

#### La versión más nueva del servible.

Enviaremos una solicitud de predicción como POST al punto de conexión REST de nuestro servidor y le pasaremos tres ejemplos. Le pediremos a nuestro servidor que nos proporcione la última versión de nuestro servible sin especificar una versión en particular.

In [ ]:
# docs_infra: no_execute
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

show(0, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  class_names[np.argmax(predictions[0])], np.argmax(predictions[0]), class_names[test_labels[0]], test_labels[0]))

#### Una versión específica del servible.

Ahora definamos una versión específica de nuestro servible. Como solo tenemos uno, seleccionemos la versión 1. También veremos los tres resultados.

In [ ]:
# docs_infra: no_execute
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

for i in range(0,3):
  show(i, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
    class_names[np.argmax(predictions[i])], np.argmax(predictions[i]), class_names[test_labels[i]], test_labels[i]))